# Predicting the Dyssynchrony Index
We will be tackling a sequence classification problem with recurrent neural networks. We believe that the vectorcardiogram is a good predictor of the dyssynchrony index, so we will treat the VCG as a sequence of coordinates and feed it into a LSTM network.

## Data Dimensions

### *Input*
Our input will be (simulated) vectorcardiograms generated by Chris Villongco using CMRG's Continuity.
Our dataset consists of 608 simulated VCGs. These simulations have varying parameters, such as differing stimulus sites and conduction velocities, but are based on the same patient (patient 2, BiV2). We limit our dataset to only 608 examples in the interest of speeding up computational time, as we are more interested in showing proof of concept than obtaining a higher accuracy (of course, we will aim for higher accuracy later).

Because we view the VCG as a sequence, each timestep can be viewed as one element in the sequence. For every time ```t```, we are given three inputs, the  ```(x, y, z)``` coordinates of the vector head.

### *Output*
We will be classifying each VCG based on the corresponding dyssynchrony index from that same simulation. The dyssynchrony index is a scalar value that theoretically ranges from 0 to 1, but we will only be concerned with the range of 0.5 to 1. This range will be further divided into 5 regular intervals, which will serve as our "classes". Specifically, these intervals will be ```[0.5, 0.6), [0.6, 0.7), [0.7, 0.8), [0.8, 0.9)```, and ```[0.9, 1.0]```. Simulations with dyssynchrony indices under 0.5 will be considered faulty and will be placed in the first interval, ```[0.5, 0.6)```. Also, note that simulations with dyssynchrony indices of 1.0 will be placed in the last interval, ```[0.9, 1.0]```. These intervals will be labeled as follows (intervals are 0-indexed):
* ```0: [0.5, 0.6)```
* ```1: [0.6, 0.7)```
* ```2: [0.7, 0.8)```
* ```3: [0.8, 0.9)```
* ```4: [0.9, 1.0]```
  
For example, a VCG sequence with a corresponding dyssynchrony index of 0.78 will be placed in class ```"2"``` since it falls in the range of 0.7 and 0.8, the third interval.

Thus, the output of the neural network will be a probability distribution signifying how likely a simulation with the given VCG will have a dyssynchrony index that falls under each of the five intervals.

## Dataset Wrapper
We've created a class that provides a basic interface for handling the dataset. Specifically, the wrapper will do the following: 
* Read in the dataset from three specified ```.npy``` files (VCG, VCG lengths, target class)
* Split the dataset into training, validation, and testing sets (the set sizes will be fixed for convenience)
* Provide a ```next_batch``` function that will return a batch of specified size for a given set.

We import the wrapper here. To instantiate, we specify the names of the NumPy files for the following:
* ```vcg.npy```: VCG sequences (input)
* ```vcg_length.npy```: VCG sequence lengths (passed as argument for ```sequence_length.npy``` parameter)
* ```target.npy```: dyssynchrony indices (target output)

In [2]:
from dataset import Patient

# Initialize dataset iterator
patient_dataset = Patient("dataset/vcg.npy", "dataset/vcg_length.npy", "dataset/target.npy")

## Initialization
### Network Dimensions
We will define the dimensions of our data, as well as the initial hyperparameters of our neural network here. Note: these parameters have not been optimized, they are simply for proof of concept.

In [3]:
# Hyperparameters
learning_rate = 0.05
training_iters = 125
batch_size = 32
num_hidden = 100
gradient_norm_threshold = 0.001


# Network Parameters
num_steps = 170
num_inputs = 3
num_classes = 5


# Where TensorFlow saves metadata for TensorBoard
logs_path='Data/'

### Input Placeholders
We define three placeholders. They are for the following:
* VCG sequence input: ```[num_steps, batch_size, num_inputs]```
* VCG sequence length: ```[batch_size]```
* targets (not hot vectors): ```[batch_size]```

In [4]:
import tensorflow as tf 

# VCG input [170, None, 3]
x = tf.placeholder(tf.float32, [num_steps, None, num_inputs])

# VCG sequence lengths
sequence_length = tf.placeholder(tf.int32, [None])

# Index of class the VCG should be categorized as
y = tf.placeholder(tf.int64, [None])

### Weights and Biases
The recurrent neural network creates an output at every timestep. Since this is a problem of sequence classification, we are only interested in the output produced at the last timestep, ```t=t_end```. We then apply a linear activation on it. The weights and biases are initialized with random values from a normal distribution, with a mean of 0.0 and a standard deviation of 1.0.

In [5]:
# Define weights and biases
# [100, 5]
weights = tf.Variable(tf.random_normal([num_hidden, num_classes]))
# [5]
biases = tf.Variable(tf.random_normal([num_classes]))

## Recurrent Neural Network Cell
Here we define what kind of recurrent neural network we will be using. We will be using a basic LSTM network with a default forget bias of 1.0, and ```tanh``` as the activation function. The ```BasicLSTMCell``` initializer function takes as parameters:
* ```num_units```: The number of units in a LSTM cell.
* ``` forget_bias```: float, the bias added to the forget gates.
* ``` activation```: activation function of the inner states. Default is ```tanh```.
* ``` state_is_tuple```: Accepted and returned states are 2-tuples of the c_state and m_state(???). Default is True

In [6]:
from tensorflow.python.ops import rnn_cell

# Define a lstm cell with tensorflow
cell = rnn_cell.BasicLSTMCell(
    num_units=num_hidden, 
    forget_bias=1.0, 
    state_is_tuple=True
)

## Prediction Operation
We will be using the ```tf.nn.dynamic_rnn``` function, instead of the ```tf.nn.rnn``` function, to get the output of the recurrent neural network. Unlike ```tf.nn.rnn```, ```tf.nn.dynamic_rnn``` takes in variable sequence lengths (it uses a ``tf.While`` loop to dynamically construct the computational graph). Also, it is faster (supposedly), despite the fact that ```tf.nn.rnn``` prebuilds the graph. The parameters are as follows:
* ```cell```: an instance of RNN cell.
* ```dtype```: (optional) The data type for the initial state and the expected output. 
* ```sequence_length```: (optional) An int32/64 vector of size [batch_size] specifying the length of each sequence.
* ```inputs```: the RNN input, a single Tensor. The dimensions are [batch_size, sequence_length, num_inputs]
* ```time_major```: Specifies that the max number of timesteps comes as the first dimensions, so the input placeholder must be of shape ```[max_time, batch_size, num_inputs]```. This requires us to permute the input matrix.


In [7]:
output, states = tf.nn.dynamic_rnn(
    cell=cell,
    dtype=tf.float32,
    sequence_length=sequence_length,
    inputs=x,
    time_major=True
)

We bring to attention the parameter ```time_major```, which we set to ```True```. This specifies that for our input placeholder, the ```max_time``` will be the *first* dimension, so it must have shape ```[max_time, batch_size, num_inputs]```. As a result, the output tensor would have shape ```[max_time, batch_size, num_hidden]```. 


In [8]:
# Shape of output tensor
print "Output tensor shape: " + str(output.get_shape())

Output tensor shape: (170, ?, 100)


This is advantageous for two reasons:
* We can easily access the *last* timestep by calling ```output[-1]``` (we are only concerned with the last timestep as this is a sequence classification problem).
* Increases efficiency because it avoids transpositions at the beginning and end of the RNN calculation (https://www.tensorflow.org/versions/r0.10/api_docs/python/nn/recurrent_neural_networks#dynamic_rnn)

However, we must alter our input, because they come in shape ```[batch_size, max_time, num_input]```, as it was the more *intuitive* way of bundling our data. Specifically, we must permute the ```0th``` and ```1st``` dimension so that the shape will be ```[max_time, batch_size, num_inputs]```. We can do so by calling NumPy's ```np.swapaxes``` function on ```batch_x``` before passing it into the ```feed_dict```.

Below is an example of permuting the ```0th``` and ```1st``` axes of the batch of VCGs:

In [9]:
import numpy as np

# Initialize a dummy iterator for the purpose of this example
dummy_dataset = Patient("dataset/vcg.npy", "dataset/vcg_length.npy", "dataset/target.npy")

# Grab the first batch
batch_x, batch_length, batch_target = dummy_dataset.train.next_batch()

# Get the shape before permutation
print "Input batch shape before swap: " + str(batch_x.shape)

# Permute the 0th and 1st axes
batch_x = np.swapaxes(batch_x, 0, 1)
print "Input batch shape after swap: " + str(batch_x.shape)

Input batch shape before swap: (32, 170, 3)
Input batch shape after swap: (170, 32, 3)


## Linear Activation
After we get the outputs for every timestep, we extract the output for the *last* timestep and apply a linear activation. 

In [10]:
logits = tf.matmul(output[-1], weights) + biases

# Shape should be [batch_size, num_classes]
print "Shape of output after linear activation: " + str(logits.get_shape())

Shape of output after linear activation: (?, 5)


## Training and Evaluation
Up to this point, we are able to feed forward our input and have the neural network output its "prediction". Now we need to set up some key functions to allow for this network to be trained.

### Cross Entropy Loss
We will be using TensorFlow's ```tf.nn.sparse_softmax_cross_entropy_with_logits``` function which measures the probability error in discrete classification tasks in which the classes are *mutually exclusive*. Note that this operation expects unscaled logits and it performs softmax internally for efficiency. 

Its parameters are as follows:
* logits: float32/64 with shape ```[batch_size, num_classes]```
* labels: int32/64 with shape ```[batch_size]``` where each entry is a value between ```[0, num_classes)```.

In [11]:
# Calculate costs for each example
costs = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, y)
print "Shape of costs (before averaging): " + str(costs.get_shape())

Shape of costs (before averaging): (?,)


This calculates the cost for each example in the batch. To find the batch cost, we average over all costs in the batch using ```tf.reduce_mean```.

Its parameters are as follows:
* input_tensor: The tensor to reduce. Should have numeric type.
* axis: The dimensions to reduce. If None (the default), reduces all dimensions.
* keep_dims: If true, retains reduced dimensions with length 1.
* name: A name for the operation (optional).

In [12]:
# Cost function
cost = tf.reduce_mean(costs)

# Output is a scalar value
print "Shape of cost (scalar value): " + str(cost.get_shape())

Shape of cost (scalar value): ()


## Optimizer
Once we define our cost function, we now know what we are trying to minimize. We will use a TensorFlow-defined operation that implements the Adam Algorithm, ```tf.train.AdamOptimizer``` with the following parameters:
* learning_rate: A Tensor or a floating point value. The learning rate (default is 0.001)
* beta1: A float value or a constant float tensor. The exponential decay rate for the 1st moment estimates (default is 0.9).
* beta2: A float value or a constant float tensor. The exponential decay rate for the 2nd moment estimates (default is 0.999).
* epsilon: A small constant for numerical stability (default is 1e-08).
* use_locking: If True use locks for update operations (default is False).
* name: Optional name for the operations created when applying gradients. Defaults to "Adam".

In [13]:
# Initialize TF optimizer
optimizer = tf.train.AdamOptimizer(
    learning_rate=learning_rate,
    name="AdamOptimizer"
)

Once we initialize an optimizer, we can simply call the ```optimizer.minimize()``` function to both calculate the gradients and apply them.

In [14]:
# Training op
train = optimizer.minimize(cost)

## Determining Accuracy
Instead of looking at the raw cost to determine the model's performance, we can simply calculate how accurately the model correctly predicted the right class that the VCG falls in. To extract the *class* that the model predicted, we simply call ```tf.argmax``` which returns the index with the largest across a specified axis of a tensor

In [15]:
# Compare predictions with targets
compare = tf.equal(tf.argmax(logits, 1), y)

# Cast booleans to ints
accuracy = tf.reduce_mean(tf.cast(compare, tf.int64))

# Shape should be a single scalar value
print "Accuracy shape (scalar value): " + str(accuracy.get_shape())

Accuracy shape (scalar value): ()


We are now ready to begin training!

In [17]:
# Initialize all the variables
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # Retrieve the entire validation set
    valid_x = patient_dataset.validate.vcg
    valid_y = patient_dataset.validate.target
    
    #for step in range(2):
    batch_x, batch_length, batch_y = patient_dataset.train.next_batch()

    sess.run(train, feed_dict={x: np.swapaxes(batch_x, 0, 1), sequence_length: batch_length, y:batch_y})